<a href="https://colab.research.google.com/github/DalilaR/LendingClub/blob/main/Prepare_LendingClub_Data221120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

From Data Exploration, we decided on features engineerings for each numerical, categorical and text features.  In here, we implemented the functions to prepare the dataset for scoring

In [ ]:
install.packages(c("caret","GGally"))
library(tidyverse)
library(caret)
library(GGally)

In [ ]:
library(tidyverse)



In [ ]:
check_and_collapse_outliers <- function(val){
  '''
  check_and_collapse_outliers:  clip outliers to the the 97.5 percentile
  val:  numerical vector 
  '''
  qt = quantile(val,probs = 0.975,na.rm= TRUE)
  pos <- which(val > qt)
  result <- val
  result[pos] <- qt
  return (result)
}

prepare_num<- function(tp_df,num_impute){
  '''
  prepare_num:  pre-process numerical features
  tp_df      :  a dataframe
  num_impute :  object to impute numerical features.  It has been prepared by
                preProcess
  result     : tp_df with the new engineered features
  
  '''
        num_cols <- unlist(lapply(tp_df, is.numeric)) 
        tp_df$total_acc[is.na(tp_df$earliest_cr_line)] <- 0
        tp_df$revol_util[is.na(tp_df$earliest_cr_line)] <- 0
        tp_df$pub_rec[is.na(tp_df$earliest_cr_line)] <- 0
        tp_df$open_acc[is.na(tp_df$earliest_cr_line)]<- 0
        tp_df$inq_last_6mths[is.na(tp_df$earliest_cr_line)]<- 0
        tp_df$delinq_2yrs[is.na(tp_df$earliest_cr_line)]<- 0
        tp_df$mths_since_last_major_derog <- factor(tp_df$mths_since_last_major_derog)

        num_cols <- unlist(lapply(tp_df, is.numeric))
        nums <- names(tp_df)[num_cols]
        ### Get the right impute.  
        #num_impute <- readRDS("impute_number.rds")
        #num_impute <- preProcess(train_data[,nums],method = ("medianImpute"))
        tp_df[,num_cols] = predict(num_impute, tp_df[,num_cols])
        tp_df$balance_open_acc  <- ifelse(tp_df$open_acc >0, tp_df$revol_bal / tp_df$open_acc,0)
        tp_df$income_balance <-  ifelse(tp_df$revol_bal >0, tp_df$annual_inc / tp_df$revol_bal,0)
        tp_df$open_total <-  ifelse(tp_df$total_acc > 0, tp_df$open_acc/ tp_df$total_acc,0)
        tp_df$pub_rec <- as.factor(tp_df$pub_rec)
        num_cols <- unlist(lapply(tp_df, is.numeric))

        nums <- names(tp_df)[num_cols]
        tp_df$revol_bal <- log(tp_df$revol_bal+1)
        tp_df$state_income <- log(tp_df$state_income)
        tp_df$balance_open_acc <- log(tp_df$balance_open_acc+1)
        tp_df[,nums] <- apply(tp_df[,nums],2, check_and_collapse_outliers)

        tp_df$emp_length <- ifelse(tp_df$emp_length>= 11,11, tp_df$emp_length)
        tp_df$open_acc <- ifelse(tp_df$open_acc >= 23, 23,tp_df$open_acc)
        tp_df$inq_last_6mths <- ifelse(tp_df$inq_last_6mths >= 4, 4,tp_df$inq_last_6mths)
        tp_df$delinq_2yrs <- ifelse(tp_df$delinq_2yrs >= 3, 3,tp_df$delinq_2yrs)
        tp_df$annual_inc <- log(tp_df$annual_inc)
        return (tp_df)
}

In [ ]:
num_impute <- readRDS("impute_number.rds")
num_impute

Created from 7780 samples and 11 variables

Pre-processing:
  - ignored (0)
  - median imputation (11)


In [ ]:
prepare_text<- function(tp_df){
  '''
  prepare_text:  create a feature Notes_na that is set to 1 if the note is empty, 
  else it is set to 0, and a feature Note_length that is the log of the length 
  of the note.  
  tp_df       :  dataframe
  return      :  dataframe with new features
  '''
  
        tp_df$Notes_na <- ifelse(is.na(tp_df$Notes),1,0)
        tp_df$Note_na <- str_replace(tp_df$Notes,"Borrower added on \\d{2}/\\d{2}/\\d{2} >","")
        tp_df$Note_length <- str_length(tp_df$Notes)
        tp_df$Note_length[is.na(tp_df$Note_length)] <- 0
        tp_df$Note_length<- log(tp_df$Note_length+1)
        return (tp_df)

}

In [ ]:
prepare_date <- function(tp_df){
  '''
  prepare_date:  derive years_of_cr_line from the difference in years between the max_date and the earliest_cr_line
  tp_df       :  dataframe
  return      :  tp_df with the new feature

  Note:  In real-life project, we would have defined the most current earliest_cr_line, 
  but in this case, for simplicity, we just assume it it the most current one
  '''

       #tp_df$earliest_cr_line <- as.Date(tp_df$earliest_cr_line,format= "%Y-%m-%d")
       max_date <- max(tp_df$earliest_cr_line,na.rm = TRUE)
       tp_df$years_of_cr_line <-  ifelse(!is.na(tp_df$earliest_cr_line),as.double(difftime(max_date ,tp_df$earliest_cr_line,units= "days"))/365,NA_real_)
       tp_df$years_of_cr_line[(tp_df$years_of_cr_line > 0) & (tp_df$years_of_cr_line < 1)] <- 1
       tp_df[,c("years_of_cr_line")] <- predict(impute_year, tp_df[,c("years_of_cr_line")])
       return (tp_df)

}

In [ ]:
prepare_categorical<-function(tp_df){
  '''
  prepare_categorical:  change some numerical features to factors, and combine
  some classes (levels) of some features
  tp_df              :  dataframe
  return             :  tp_df with the new features
  '''

        to_factor <- c("emp_length","open_acc","inq_last_6mths","delinq_2yrs","mths_since_last_major_derog")
        tp_df[,to_factor] <- lapply(tp_df[to_factor], as.factor) 

        tp_df <- tp_df[tp_df$home_ownership != "NONE",]
        tp_df$home_ownership <- droplevels(tp_df$home_ownership)
        purpose_table <- read_csv("purpose_lookup_table.csv")
        purpose_table <- purpose_table %>% column_to_rownames(., var = "cat")
        tp_df$purpose_cat_clean <- purpose_table$new_cat[tp_df$purpose_cat]
        tp_df$purpose_cat_clean <- as.factor(tp_df$purpose_cat_clean)
        print(dim(tp_df))
        return(tp_df)
}

In [ ]:
train_data <- read_csv("training_data_ready.csv")

In [ ]:

test_data <- read_csv("test_data.csv")

In [ ]:

factor_cols <- c("home_ownership","verification_status","purpose_cat","policy_code")
test_data[,factor_cols] <- lapply(test_data[,factor_cols],as.factor)
tp_df <- prepare_date(test_data)
tp_df <- prepare_num(tp_df,num_impute)
tp_df <- prepare_categorical(tp_df)
tp_df <- prepare_text(tp_df)

write_csv(tp_df,"test_data_ready221120.csv")

In [ ]:
str(tp_df)

tibble [1,999 × 36] (S3: tbl_df/tbl/data.frame)
 $ Id                         : num [1:1999] 5 7 10 20 24 32 33 36 39 57 ...
 $ is_bad                     : num [1:1999] 0 0 0 0 0 0 0 0 0 0 ...
 $ emp_title                  : chr [1:1999] "Belmont Correctional" "Peninsula Counseling Center" NA "Office of the Federal Defender" ...
 $ emp_length                 : Factor w/ 10 levels "1","2","3","4",..: 10 10 1 3 1 3 5 2 3 3 ...
 $ home_ownership             : Factor w/ 4 levels "MORTGAGE","OTHER",..: 1 1 4 1 1 1 4 4 3 3 ...
 $ annual_inc                 : num [1:1999] 50004 126000 40000 120000 191000 ...
 $ verification_status        : Factor w/ 3 levels "not verified",..: 2 1 1 3 1 2 1 2 1 2 ...
 $ pymnt_plan                 : chr [1:1999] "n" "n" "n" "n" ...
 $ Notes                      : chr [1:1999] "I want to consolidate my debt, pay for a vacation and buy a ring." "Borrower added on 05/18/10 > mick credit card consolidation loan - 100% payoff of credit card debt - amex, sear"| __t